Мышковец С.А., v.1 1.05.2023

Решение задачи:

Обучить модель на основе hugging faces transformers на отзывах из amazon для дальнейшей оценки качества работы разных методов саммаризации.


In [270]:
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np 

import tensorflow as tf 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score
from sklearn.utils import shuffle

import evaluate

import seaborn as sns 
plt.style.use('ggplot')

import gc

---

In [1]:
from datasets import load_dataset

spanish_dataset = load_dataset("amazon_reviews_multi", "es")
english_dataset = load_dataset("amazon_reviews_multi", "en")
english_dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/200000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5000 [00:00<?, ? examples/s]

...

Generating validation split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Dataset amazon_reviews_multi downloaded and prepared to /Users/sviatlanamyshkavets/.cache/huggingface/datasets/amazon_reviews_multi/en/1.0.0/724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 200000
    })
    validation: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 5000
    })
})

In [2]:
def show_samples(dataset, num_samples=3, seed=42):
    sample = dataset["train"].shuffle(seed=seed).select(range(num_samples))
    for example in sample:
        print(f"\n'>> Title: {example['review_title']}'")
        print(f"'>> Review: {example['review_body']}'")


show_samples(english_dataset)


'>> Title: Worked in front position, not rear'
'>> Review: 3 stars because these are not rear brakes as stated in the item description. At least the mount adapter only worked on the front fork of the bike that I got it for.'

'>> Title: meh'
'>> Review: Does it’s job and it’s gorgeous but mine is falling apart, I had to basically put it together again with hot glue'

'>> Title: Can't beat these for the money'
'>> Review: Bought this for handling miscellaneous aircraft parts and hanger "stuff" that I needed to organize; it really fit the bill. The unit arrived quickly, was well packaged and arrived intact (always a good sign). There are five wall mounts-- three on the top and two on the bottom. I wanted to mount it on the wall, so all I had to do was to remove the top two layers of plastic drawers, as well as the bottom corner drawers, place it when I wanted and mark it; I then used some of the new plastic screw in wall anchors (the 50 pound variety) and it easily mounted to the wall. 

In [3]:
english_dataset.set_format("pandas")
english_df = english_dataset["train"][:]
# Show counts for top 20 products
english_df["product_category"].value_counts()[:20]

home                      17679
apparel                   15951
wireless                  15717
other                     13418
beauty                    12091
drugstore                 11730
kitchen                   10382
toy                        8745
sports                     8277
automotive                 7506
lawn_and_garden            7327
home_improvement           7136
pet_products               7082
digital_ebook_purchase     6749
pc                         6401
electronics                6186
office_product             5521
shoes                      5197
grocery                    4730
book                       3756
Name: product_category, dtype: int64

In [84]:
def filter_books(example):
    return (
        example["product_category"] == "book"
        or example["product_category"] == "digital_ebook_purchase"
    )

In [85]:
english_dataset.reset_format()

In [86]:
spanish_books = spanish_dataset.filter(filter_books)
english_books = english_dataset.filter(filter_books)
show_samples(english_books)

Filter:   0%|          | 0/200000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [87]:
from datasets import concatenate_datasets, DatasetDict

books_dataset = DatasetDict()

for split in english_books.keys():
    books_dataset[split] = concatenate_datasets(
        [english_books[split], spanish_books[split]]
    )
    books_dataset[split] = books_dataset[split].shuffle(seed=42)

# Peek at a few examples
show_samples(books_dataset)

In [88]:
books_dataset = books_dataset.filter(lambda x: len(x["review_title"].split()) > 2)

Filter:   0%|          | 0/17612 [00:00<?, ? examples/s]

Filter:   0%|          | 0/424 [00:00<?, ? examples/s]

Filter:   0%|          | 0/442 [00:00<?, ? examples/s]

In [89]:
books_dataset

DatasetDict({
    train: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 9672
    })
    validation: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 238
    })
    test: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 245
    })
})

In [9]:
from transformers import AutoTokenizer

model_checkpoint = "google/mt5-small"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/opt/homebrew/Caskroom/miniforge/base/envs/tf_mac/lib/python3.9/site-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [90]:
inputs = tokenizer("I loved reading the Hunger Games!")
inputs

{'input_ids': [336, 259, 28387, 11807, 287, 62893, 295, 12507, 309, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [91]:
tokenizer.convert_ids_to_tokens(inputs.input_ids)

['▁I', '▁', 'loved', '▁reading', '▁the', '▁Hung', 'er', '▁Games', '!', '</s>']

In [92]:
max_input_length = 512
max_target_length = 30


def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["review_body"],
        max_length=max_input_length,
        truncation=True,
    )
    labels = tokenizer(
        examples["review_title"], max_length=max_target_length, truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [93]:
tokenized_datasets = books_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/9672 [00:00<?, ? examples/s]

Map:   0%|          | 0/238 [00:00<?, ? examples/s]

Map:   0%|          | 0/245 [00:00<?, ? examples/s]

In [58]:
generated_summary = "I absolutely loved reading the Hunger Games"
reference_summary = "I loved reading the Hunger Games"

In [ ]:
# !pip install rouge_score

In [59]:
import evaluate

rouge_score = evaluate.load("rouge")

In [94]:
scores = rouge_score.compute(
    predictions=[generated_summary], references=[reference_summary]
)
scores

{'rouge1': 0.923076923076923,
 'rouge2': 0.7272727272727272,
 'rougeL': 0.923076923076923,
 'rougeLsum': 0.923076923076923}

In [ ]:
# !pip install nltk

In [61]:
import nltk

nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sviatlanamyshkavets/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [95]:
from nltk.tokenize import sent_tokenize


def three_sentence_summary(text):
    return "\n".join(sent_tokenize(text)[:3])


print(three_sentence_summary(books_dataset["train"][1]["review_body"]))

In [96]:
def evaluate_baseline(dataset, metric):
    summaries = [three_sentence_summary(text) for text in dataset["review_body"]]
    return metric.compute(predictions=summaries, references=dataset["review_title"])

In [97]:
import pandas as pd

score = evaluate_baseline(books_dataset["validation"], rouge_score)
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_dict = dict((rn, round(score[rn] * 100, 2)) for rn in rouge_names)
rouge_dict

{'rouge1': 16.74, 'rouge2': 8.8, 'rougeL': 15.55, 'rougeLsum': 15.93}

In [31]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [41]:
from huggingface_hub import notebook_login

notebook_login()

In [98]:
from transformers import Seq2SeqTrainingArguments

batch_size = 8
num_train_epochs = 8
# Show the training loss with every epoch
logging_steps = len(tokenized_datasets["train"]) // batch_size
model_name = model_checkpoint.split("/")[-1]

args = Seq2SeqTrainingArguments(
    output_dir=f"{model_name}-finetuned-amazon-en-es",
    evaluation_strategy="epoch",
    learning_rate=5.6e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=num_train_epochs,
    predict_with_generate=True,
    logging_steps=logging_steps,
    push_to_hub=True,
)

In [99]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Decode generated summaries into text
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # Decode reference summaries into text
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # ROUGE expects a newline after each sentence
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    # Compute ROUGE scores
    result = rouge_score.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    # Extract the median scores
    result = {key: value * 100 for key, value in result.items()}
    return {k: round(v, 4) for k, v in result.items()}

In [100]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [102]:
tokenized_datasets = tokenized_datasets.remove_columns(
    books_dataset["train"].column_names
)

In [103]:
features = [tokenized_datasets["train"][i] for i in range(2)]
data_collator(features)

{'input_ids': tensor([[   653,   1957,   1314,    261,   2757,   1280,    435,    259,  29166,
            263,    269,    774,   5547,      1,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              

In [104]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/Users/sviatlanamyshkavets/Documents/Courses/DS TM/Final/mt5-small-finetuned-amazon-en-es is already a clone of https://huggingface.co/solmysh/mt5-small-finetuned-amazon-en-es. Make sure you pull the latest changes with `repo.git_pull()`.


In [105]:
trainer.train()

/opt/homebrew/Caskroom/miniforge/base/envs/tf_mac/lib/python3.9/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/9672 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

...

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

TrainOutput(global_step=9672, training_loss=3.2002855021663694, metrics={'train_runtime': 151341.1725, 'train_samples_per_second': 0.511, 'train_steps_per_second': 0.064, 'train_loss': 3.2002855021663694, 'epoch': 8.0})

In [162]:
trainer.save_model()

To https://huggingface.co/solmysh/mt5-small-finetuned-amazon-en-es
   0e3cb17..28accaf  main -> main

   0e3cb17..28accaf  main -> main



In [106]:
trainer.evaluate()

  0%|          | 0/30 [00:00<?, ?it/s]

{'eval_loss': 3.0134811401367188,
 'eval_rouge1': 16.5421,
 'eval_rouge2': 7.9012,
 'eval_rougeL': 16.2574,
 'eval_rougeLsum': 16.1537,
 'eval_runtime': 36.7312,
 'eval_samples_per_second': 6.479,
 'eval_steps_per_second': 0.817,
 'epoch': 8.0}

In [107]:
trainer.push_to_hub(commit_message="Training complete", tags="summarization")

Several commits (3) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 1.00/1.12G [00:00<?, ?B/s]

Upload file runs/May16_18-07-05_MacBook-Pro-Sviatlana.local/events.out.tfevents.1684249648.MacBook-Pro-Sviatla…

Upload file runs/May16_18-07-05_MacBook-Pro-Sviatlana.local/events.out.tfevents.1684401170.MacBook-Pro-Sviatla…

To https://huggingface.co/solmysh/mt5-small-finetuned-amazon-en-es
   0736f31..ad68cb4  main -> main

   0736f31..ad68cb4  main -> main

To https://huggingface.co/solmysh/mt5-small-finetuned-amazon-en-es
   ad68cb4..0e3cb17  main -> main

   ad68cb4..0e3cb17  main -> main



'https://huggingface.co/solmysh/mt5-small-finetuned-amazon-en-es/commit/ad68cb48230fd40e096d8ff66ff94dcf810042ce'

In [261]:
from transformers import pipeline

hub_model_id = "huggingface-course/mt5-small-finetuned-amazon-en-es"
summarizer = pipeline("summarization", model=hub_model_id, max_length=20)

In [262]:
def print_summary(idx):
    review = books_dataset["test"][idx]["review_body"]
    title = books_dataset["test"][idx]["review_title"]
    summary = summarizer(books_dataset["test"][idx]["review_body"])[0]["summary_text"]
    return f"Review: {review}" , f"Title: {title}", f"Summary: {summary}"

In [263]:
print_summary(199)

('Review: Good, but not his best. Well-researched but basically revolves around the growth of one town in Ohio. Would have liked broader view',
 'Title: Good, but not his best.',
 'Summary: Good, but not his best')

In [264]:
summarizer("""I bought this book because I read some glowing praise on an online library site. Unfortunately, I was deeply disappointed by page three. I always buy books in the hope and expectation of having an enjoyable read, not to criticise. However, this book is in urgent need of good editing -- though quite possibly editing alone wouldn't save it. Examples: a bed squeaks slightly and sharply in the same sentence; a nightgown hangs freely over her girlish figure and olive colored complexion; coffee aromas huddle; rumbling clouds huddle (as well as the coffee?); she prepared to sip her coffee beneath the wrath of God... cuddled within the arms of a strong breeze; (the wrath of God is a breeze?); the Columbian (stet) coffee aroma danced beneath her sculpted tan nose; the coffee bean fragrance tangoed within her body; she placed her thick pink lips against the warm cup;... and so on, all by page three. It is quite possible that the storyline is deeply moving. I'll never know because I can't bring myself to continue. Sorry.""")

[{'summary_text': 'Very disappointed'}]

In [265]:
summarizer("""This book is one of my all time favorite Garwood novels because it features one of my all time favorite characters - Brodich Buchanan. Brodich is most definitely an alpha male with a wry sense of humor. The supporting characters add to the fun. The only downside to the book was the rushed treatment that Ramsey received. The secondary romance was a bit weak for my taste. Even so, I gave the review 5 stars because Brodich is that good.""")

[{'summary_text': 'Very good'}]

In [164]:
df = pd.read_csv('train_df.csv')                  

In [165]:
df

,score,summary,review
0,1.0,not good,I bought this book because I read some glowing...
1,1.0,Buyer beware,"This is a self-published book, and if you want..."
2,1.0,The Worst!,A complete waste of time. Typographical errors...
3,1.0,Oh please,I guess you have to be a romance novel lover f...
4,1.0,Awful beyond belief!,I feel I have to write to keep others from was...
...,...,...,...
24995,5.0,You have to read this!,&quot;The Chosen&quot; is one of my overall fa...
24996,5.0,More Night World,"I can read these books over and over, but espe..."
24997,5.0,This was one of the best in the series!!!!!,Rashel was definitely a heroic character and s...
24998,5.0,Fantastic,This was the first Night World book that I eve...


In [ ]:
# prepare data for df['predicted summary']

predicted_summary = []
for i in df['review']:
    summary = summarizer(i)
    predicted_summary.append(summary)

In [216]:
values = []
for i in predicted_summary:
    for e in i:
        for k, v in e.items():
            value = v
            values.append(value)

In [226]:
df['model predicted summary'] = values

In [230]:
df = df.drop(['model predicted summaty'], axis=1)

In [238]:
df.columns

Index(['score', 'summary', 'review', 'model predicted summary'], dtype='object')

In [271]:
df_shuffle = shuffle(df)

In [273]:
df_shuffle.tail(30)

,score,summary,review,model predicted summary
24233,5.0,A detailed exploration into the issues of fema...,"Expertly written by Paxton Quigly, ""Stayin' Al...",Very detailed exploration into the issues of f...
18942,4.0,Lschs student review for five points extra cre...,The book A Philadelphia Catholic in King James...,Another thing I liked in the story outweighed ...
8666,2.0,Sorry Poul,"Oh Poul. When it came to big ideas, he was unb...","Very interesting, but a few pages read like an..."
6396,2.0,Limited combat involvement,`A Paratrooper's Panoramic View' by Robert L. ...,Very useful
19769,4.0,A Good Read,"A very good book, hard to put down. I was hopi...","A very good book, hard to put down"
20939,5.0,Understanding the philosophy of economics,These concepts are important for everyone to f...,Very easy to read
17568,4.0,Hard Times is a harder read,While this was a faster paced book than usual ...,Very boring
6420,2.0,"Interesting premise, but biased and repetitive","This book's strong point, as other readers hav...","Very interesting, but a very interesting read."
5051,2.0,"So far, the worst book of one of my favorite a...","Having read all of Patricia Cornwell's books, ...",Patricia Cornwell's books were described incon...
5311,2.0,The autobiography of my mother,Did not really get lost in there book....found...,boring


In [247]:
df[4100:4105]

,score,summary,review,model predicted summary
4100,1.0,wrong book,They sent me the wrong book. It was The Berens...,They sent me the wrong book
4101,1.0,The Only Good Thing About It Is The Title &amp...,"Being unnaturally obsessed with Kafka, it was ...",Very interesting
4102,1.0,tosses out morality and rationality.......,The Stranger is an existential story of a man ...,Very interesting read
4103,1.0,A horrible translation,I have read a previous translation of The Stra...,Very moving
4104,1.0,Do you really like existentialist atheism and ...,This book brought me a chilling and troubling ...,Another interesting read


In [248]:
for i in df[4100:4105].review:
    display(i)

'They sent me the wrong book. It was The Berenstain Bears, Learn About Strangers, instead of The Stranger. It was disappointing!!'

'Being unnaturally obsessed with Kafka, it was assumed by my compadres that I would also like this author, and particularly this book. How wrong they were. The plot makes no sense, the characterization is meager &amp; unconvincing, and the only poetic prose occurs on the last page as our hero (ha!) languishes in prison in the final hours before he is to be executed, wondering what it all means &amp; railing at humanity for treating him like the dog he is.'

'The Stranger is an existential story of a man living his life as a self-determing individual. He feels that nothing is truly wrong or right and therefore nothing that he does makes any difference or has any real impact except to himself. It is almost as if he is on a stage writing his life as he goes, acting it out and the rest of the universe is merely a backdrop for whatever he chooses to pursue. It is a timeless novel that tosses out morality and rationality as a means to make choices. It shows a vast emptiness, a soulessness that is overpowering in its totality. This is an amazing, insightful look at life where no matter what one does, the end will always be the same. It is interesting to note that Camus while having the belief that the universe was a cold and indifferent place, still believed in the dignity of each individual being and had a concern for their welfare. It is one of the most widely read books, but it is bleak in an eye-opening manner.'

'I have read a previous translation of The Stranger, and was deeply moved. My entire life was changed. The previous translator did Camus justice. Matthew Ward, with this translation of The Stranger, ruined the novel. Ward includes awful cliche and unintelligent description. Unfortunately this is the only translation currently in print in the US. If you are able to, please order from a forgein printer (sometimes printed under the title The Outsider) or consider searching for an out of print copy not translated by Matthew Ward.'

"This book brought me a chilling and troubling nightmare of what it means to live without hope, which according to Camus, is a positive expression of the freedom of pushing a mute rock up a hill forever in joy as he explicates in his discursive book, The Myth of Sisyphus. But when a spiritual and religious person reads this 'fiction', she will recognize the truth that Camus' work never ceases to entrap itself in death-glorification, even, as in the final paragraph of this book, the stranger finds himself 'as having been happy and happy still', at the thought of the advent of his death, taking joy in the expectation (which, ironically, is a betrayal of Camus' conception of loss of hope as being a positive thing) of the spectacle of his death of execution and the jeers of the mob of spectators as they see him killed. The 'protagonist' of The Stranger is a cold-hearted emotionless character, and throughout this whole book of 'fiction' he turns the stomach. Read it only if you want to be t

In [249]:
for i in df[4100:4105]['model predicted summary']:
    display(i)

'They sent me the wrong book'

'Very interesting'

'Very interesting read'

'Very moving'

'Another interesting read'

In [228]:
df.to_csv('/Users/sviatlanamyshkavets/Documents/Courses/DS TM/Final/model_predicted_summary_train.csv', index=False)

---

In [266]:
summarizer("""Residents of a tiny Swiss village have all been evacuated because of the risk of an imminent rockslide.

Brienz's fewer than 100 villagers were given just 48 hours to pack what they could and abandon their homes.

Even the dairy cows were loaded up for departure after geologists warned a rockfall was imminent.

Two million cubic metres of rock is coming loose from the mountain above, and a rockslide could obliterate the village.

The development has raised questions about the safety of some mountain communities, as global warming changes the alpine environment.

Brienz, in the eastern canton of Graubünden, is now empty.

The village has been judged a geological risk for some time and is built on land that is subsiding down towards the valley, causing the church spire to lean and large cracks to appear in buildings.

As the minutes ticked towards the deadline to leave, even Brienz's dairy cows were being taken to safety.

The residents, some young, some old, families, farmers and professional couples, had two days to abandon their homes.

They were asked earlier this week to evacuate the village by Friday evening.

Switzerland's Alpine regions are especially sensitive to global warming - as the permafrost high in the mountains begins to thaw, the rock becomes more unstable.

This particular mountain has always been unstable, but recently the rock has been shifting faster and faster.

Days of heavy rain could bring two million cubic metres of loosened rock crashing down the mountainside onto the village, scientists warned.

Now the villagers must wait, in temporary accommodation, for the rock to fall - and hope it misses their homes.""")

[{'summary_text': 'Accidental rockslide'}]

In [267]:
summarizer("""Waiter: Hello, good evening. Can I start you off with some refreshing drink?

Rana: Yes. I’ll have iced tea, please.

Amal: And I’ll have a chocolate cold coffee.

Waiter: Ok. Should I take your order now, or do you need a few minutes more?

Rana: No no we are ready, you can take the order. I’ll have the corn mushroom soup to start, and the grilled chicken with mashed potatoes and peas. And, please also bring a bowl of garlic rice.

Waiter: Sure sir. How do you want the chicken— low spicy, medium, or high on spice?

Rana: Medium spice, please.

Amal: And I’ll just have the beef, with bread and a salad.""")

[{'summary_text': 'Very good evening. Can I start you off with some refreshing drink?'}]